In [315]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, accuracy_score, r2_score
from sklearn.impute import SimpleImputer

In [316]:
train_set = pd.read_csv('./train.csv')
test_set = pd.read_csv('./test.csv')
train = train_set.copy()
test = test_set.copy()


In [317]:
y = train_set.SalePrice
col = train_set.columns.to_list()
col.remove('SalePrice')
train = train_set[col]

Xử lý missing data

Ghi lại những cột có missing data

In [318]:
have_na = []
for col in train.columns :
    if train[col].isnull().sum() > 0 :
        have_na.append(col)

Tìm tỉ lệ dữ liệu bị mất của từng cột

In [319]:
missing_prc = {}
for col in have_na : 
    missing_prc[col]= train[col].isnull().sum() / len(train[col].index)
    print (f"cot {col} co {np.around((missing_prc[col])*100, 2)}% missing data")

cot LotFrontage co 17.74% missing data
cot Alley co 93.77% missing data
cot MasVnrType co 59.73% missing data
cot MasVnrArea co 0.55% missing data
cot BsmtQual co 2.53% missing data
cot BsmtCond co 2.53% missing data
cot BsmtExposure co 2.6% missing data
cot BsmtFinType1 co 2.53% missing data
cot BsmtFinType2 co 2.6% missing data
cot Electrical co 0.07% missing data
cot FireplaceQu co 47.26% missing data
cot GarageType co 5.55% missing data
cot GarageYrBlt co 5.55% missing data
cot GarageFinish co 5.55% missing data
cot GarageQual co 5.55% missing data
cot GarageCond co 5.55% missing data
cot PoolQC co 99.52% missing data
cot Fence co 80.75% missing data
cot MiscFeature co 96.3% missing data


In [320]:
cols_to_drop = [col for col in have_na if missing_prc[col] >= 0.4]

train = train.drop(columns=cols_to_drop)
test = test.drop(columns=cols_to_drop)

# cập nhật have_na
have_na = [col for col in have_na if col not in cols_to_drop]


Simple Imputer cho những cột số còn lại
 :


In [321]:
SI = SimpleImputer(strategy='mean')
SI.fit(train.select_dtypes(exclude='object'))
temp_train = SI.transform(train.select_dtypes(exclude='object')) 
temp_test = SI.transform(test.select_dtypes(exclude='object'))
temp_train = pd.DataFrame(temp_train, columns=train.select_dtypes(exclude='object').columns)
train[temp_train.columns] = temp_train
temp_test = pd.DataFrame(temp_test, columns=test.select_dtypes(exclude='object').columns)
test[temp_test.columns] = temp_test

In [322]:
for col in have_na : 
    missing_prc[col]= train[col].isnull().sum() / len(train[col].index)
    print (f"cot {col} co {np.around((missing_prc[col])*100, 2)}% missing data")

cot LotFrontage co 0.0% missing data
cot MasVnrArea co 0.0% missing data
cot BsmtQual co 2.53% missing data
cot BsmtCond co 2.53% missing data
cot BsmtExposure co 2.6% missing data
cot BsmtFinType1 co 2.53% missing data
cot BsmtFinType2 co 2.6% missing data
cot Electrical co 0.07% missing data
cot GarageType co 5.55% missing data
cot GarageYrBlt co 0.0% missing data
cot GarageFinish co 5.55% missing data
cot GarageQual co 5.55% missing data
cot GarageCond co 5.55% missing data


lướt qua mỗi cột, nếu cột đó trong train có giá trị *null*, tạo cột *[col was missing]* có giá trị là *Series true/false* do train[col].isnull() trả về  


***CATEGORICAL VARIABLES***  
Là dạng dữ liệu chỉ có số lượng giá trị nhất định : (đắt, rẻ), (1 tầng, 2 tầng, 3 tầng), (giàu, nghèo), (starbucks, mcdonalds, lotteria)

Tách cột có dữ dạng object ra 

In [323]:
train_cate = train.select_dtypes(include = ['object'])
test_cate = test.select_dtypes(include = ['object'])

In [324]:
train_cate.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,SBrkr,Gd,Typ,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal


Phân loại categorical và text thường

In [325]:
cate_col = [col for col in train_cate.columns if (train_cate[col].nunique()<=6 & train_cate[col].nunique() >2)]

In [326]:
train_cate[cate_col]

,LotShape,LandContour,RoofStyle,ExterQual,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,KitchenQual,GarageType,SaleCondition
0,Reg,Lvl,Gable,Gd,PConc,Gd,TA,No,GLQ,Unf,GasA,Gd,Attchd,Normal
1,Reg,Lvl,Gable,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,TA,Attchd,Normal
2,IR1,Lvl,Gable,Gd,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Gd,Attchd,Normal
3,IR1,Lvl,Gable,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Detchd,Abnorml
4,IR1,Lvl,Gable,Gd,PConc,Gd,TA,Av,GLQ,Unf,GasA,Gd,Attchd,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Reg,Lvl,Gable,TA,PConc,Gd,TA,No,Unf,Unf,GasA,TA,Attchd,Normal
1456,Reg,Lvl,Gable,TA,CBlock,Gd,TA,No,ALQ,Rec,GasA,TA,Attchd,Normal
1457,Reg,Lvl,Gable,Ex,Stone,TA,Gd,No,GLQ,Unf,GasA,Gd,Attchd,Normal
1458,Reg,Lvl,Hip,TA,CBlock,TA,TA,Mn,GLQ,Rec,GasA,Gd,Attchd,Normal


Ordinal Encoder cho SaleCondition


In [327]:
OE = OrdinalEncoder()
OE.fit(train[['SaleCondition']])
train['SaleCondition'] = OE.transform(train[['SaleCondition']])
test['SaleCondition'] = OE.transform(test[['SaleCondition']])

Hiện tại :  
train : train set chính, cột SaleCondition được OrdinalEncoded, các cột numeric được imputed, cột categorical khác thì chưa -> cần fillna cho categorical ones, standard scale (không thật sự cần) cho numeric ones, feature selection (test) -> one hot encoder rồi train model

In [328]:
train_set.corr(numeric_only=True).abs()['SalePrice']

Id               0.021917
MSSubClass       0.084284
LotFrontage      0.351799
LotArea          0.263843
OverallQual      0.790982
OverallCond      0.077856
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.477493
BsmtFinSF1       0.386420
BsmtFinSF2       0.011378
BsmtUnfSF        0.214479
TotalBsmtSF      0.613581
1stFlrSF         0.605852
2ndFlrSF         0.319334
LowQualFinSF     0.025606
GrLivArea        0.708624
BsmtFullBath     0.227122
BsmtHalfBath     0.016844
FullBath         0.560664
HalfBath         0.284108
BedroomAbvGr     0.168213
KitchenAbvGr     0.135907
TotRmsAbvGrd     0.533723
Fireplaces       0.466929
GarageYrBlt      0.486362
GarageCars       0.640409
GarageArea       0.623431
WoodDeckSF       0.324413
OpenPorchSF      0.315856
EnclosedPorch    0.128578
3SsnPorch        0.044584
ScreenPorch      0.111447
PoolArea         0.092404
MiscVal          0.021190
MoSold           0.046432
YrSold           0.028923
SalePrice        1.000000
Name: SalePr

In [329]:
corr = [col for col in train_set.corr(numeric_only=True).abs()['SalePrice'].index if train_set.corr(numeric_only=True).abs()['SalePrice'][col] >= 0.2]

In [330]:
corr.remove('SalePrice')

In [331]:
feature_train = train[corr + cate_col]
feature_train.fillna('M')
feature_test = test[corr + cate_col]
feature_test.fillna('M')

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,KitchenQual,GarageType,SaleCondition
0,80.0,11622.0,5.0,1961.0,1961.0,0.0,468.0,270.0,882.0,896.0,...,CBlock,TA,TA,No,Rec,LwQ,GasA,TA,Attchd,4.0
1,81.0,14267.0,6.0,1958.0,1958.0,108.0,923.0,406.0,1329.0,1329.0,...,CBlock,TA,TA,No,ALQ,Unf,GasA,Gd,Attchd,4.0
2,74.0,13830.0,5.0,1997.0,1998.0,0.0,791.0,137.0,928.0,928.0,...,PConc,Gd,TA,No,GLQ,Unf,GasA,TA,Attchd,4.0
3,78.0,9978.0,6.0,1998.0,1998.0,20.0,602.0,324.0,926.0,926.0,...,PConc,TA,TA,No,GLQ,Unf,GasA,Gd,Attchd,4.0
4,43.0,5005.0,8.0,1992.0,1992.0,0.0,263.0,1017.0,1280.0,1280.0,...,PConc,Gd,TA,No,ALQ,Unf,GasA,Gd,Attchd,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,21.0,1936.0,4.0,1970.0,1970.0,0.0,0.0,546.0,546.0,546.0,...,CBlock,TA,TA,No,Unf,Unf,GasA,TA,M,4.0
1455,21.0,1894.0,4.0,1970.0,1970.0,0.0,252.0,294.0,546.0,546.0,...,CBlock,TA,TA,No,Rec,Unf,GasA,TA,CarPort,0.0
1456,160.0,20000.0,5.0,1960.0,1996.0,0.0,1224.0,0.0,1224.0,1224.0,...,CBlock,TA,TA,No,ALQ,Unf,GasA,TA,Detchd,0.0
1457,62.0,10441.0,5.0,1992.0,1992.0,0.0,337.0,575.0,912.0,970.0,...,PConc,Gd,TA,Av,GLQ,Unf,GasA,TA,M,4.0


In [332]:
cate_col2 = [col for col in cate_col if col!='SaleCondition']

In [333]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(feature_train, y, test_size=0.2)

In [334]:
type(x_train)

pandas.core.frame.DataFrame

In [335]:
CT = ColumnTransformer(transformers = [('encode', OneHotEncoder(handle_unknown='ignore'), cate_col2)], remainder = 'passthrough')
x_train = CT.fit_transform(x_train)
x_test = CT.transform(x_test)


In [336]:
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor (max_depth=7, max_leaf_nodes=250)
RFR.fit(x_train, y_train) 
prediction = RFR.predict(x_test)
result = pd.DataFrame({'Real' : y_test,
                      'Prediction' : prediction})

In [337]:
valid = mean_absolute_error(prediction, y_test) 

In [338]:
valid

19828.909186059387

In [339]:
result

,Real,Prediction
89,123600,113582.143986
840,140000,140994.098554
697,123500,122466.546458
195,148500,163242.657495
1352,134900,118021.628216
...,...,...
463,188700,156024.163035
986,117000,129937.953900
257,220000,235013.698040
119,163990,174669.619992


In [340]:
result.to_csv('result.csv', index = False)